In [37]:
%%writefile distribute_2peptide_2_96well.py

from opentrons import protocol_api
import time
import json

# metadata
metadata = {
    'protocolName': 'Distribute Peptides 384',
    'author': 'Colin Rathbun <rathbunc@dickinson.edu>',
    'description': 'Distribute a 96 well plate plus 2 substrates into two, 96 well plates.',
    'apiLevel': '2.12'
}

# for now, this only works with two peptides
PEPTIDE_WELLS = ['A1','A2']
FRZ_WELL = 'A12'

# protocol run function. the part after the colon lets your editor know
# where to look for autocomplete suggestions
def run(protocol: protocol_api.ProtocolContext):
    amounts = {
        'peptides':{
            1:0,
            2:0,
        },
        'frz':0
    }

    # labware
    # load custom plates?
    deepwell_def = json.load(open('../labware/labcon_96_wellplate_2200ul/labcon_96_wellplate_2200ul.json'))
    #use this one for now
    celltreat96_def = json.load(open('../labware/celltreat_96_wellplate_350ul/celltreat_96_wellplate_350ul.json'))

    deepwell_plates = [
    protocol.load_labware_from_definition(deepwell_def, 1),
    protocol.load_labware_from_definition(deepwell_def, 2),
    ]
    well96_plates = [
        protocol.load_labware_from_definition(celltreat96_def, 3),
        protocol.load_labware_from_definition(celltreat96_def, 4),
    ]
    
    well12 = protocol.load_labware('nest_12_reservoir_15ml', 5)

    tip_racks = [
        protocol.load_labware('opentrons_96_tiprack_20ul', 11),
        protocol.load_labware('opentrons_96_tiprack_20ul', 10),
        protocol.load_labware('opentrons_96_tiprack_20ul', 9),
        protocol.load_labware('opentrons_96_tiprack_20ul', 8),
        protocol.load_labware('opentrons_96_tiprack_20ul', 7),
        protocol.load_labware('opentrons_96_tiprack_20ul', 6),
    ]

    # pipettes
    right_pipette = protocol.load_instrument('p20_multi_gen2', 'right', tip_racks=tip_racks)

    # Add peptides to wells first.
    for n, well in enumerate(PEPTIDE_WELLS):
        transfer_amount = 20
        right_pipette.transfer(transfer_amount, well12.wells_by_name()[well], well96_plates[n].rows())
        amounts['peptides'][n+1] += transfer_amount*8*len(well96_plates[n].rows())
    
    # Now distribute the two, 96 well plates of mutants into the 96 blackwell plates.
    for n, plate in enumerate(deepwell_plates):
        for i, col in enumerate(plate.columns_by_name()):
            right_pipette.transfer(
                10, 
                plate.columns_by_name()[col], 
                well384.columns()[i + (12*n)], # so that the second plate is on the right side of the 384
                mix_before=(3,20), 
                mix_after=(3,20),
                new_tip='always'
                )
    
    # Wait 5 minutes for things to fully mix
    # Maybe this is when we can add tips to the deck??
    incubate_start = time.perf_counter()
    protocol.pause("Please add tips to any empty positions.")
    incubate_end = time.perf_counter()

    # This assumes that we added tips!!
    right_pipette.reset_tipracks()

    # Add additional delay if tips were replaced too quickly
    if incubate_end - incubate_start < 300:
        protocol.delay(seconds=300-(incubate_end-incubate_start))

    # Now add FRZ
    right_pipette.transfer(5, well12.wells_by_name()[FRZ_WELL], well384.wells(),
        new_tip='always',
        mix_after=(3,20)
    )
    amounts['frz']+=384*5
    
    print("Amounts of reagents used: ",amounts)


Overwriting distribute_2peptide_384well.py


In [38]:
from opentrons.simulate import simulate, format_runlog

log, _bundle = simulate(open('./distribute_2peptide_384well.py'))

print(format_runlog(log))

Grenier BIO-ONE 384 Well Plate 138 µL on 3
transferred:  0 A1
transferred:  1 A2
Amounts of reagents used:  {'peptides': {1: 1920, 2: 1920}, 'frz': 1920}
Transferring 10.0 from A1 of NEST 12 Well Reservoir 15 mL on 4 to A1 of Grenier BIO-ONE 384 Well Plate 138 µL on 3
	Picking up tip from A1 of Opentrons 96 Tip Rack 20 µL on 11
	Aspirating 10.0 uL from A1 of NEST 12 Well Reservoir 15 mL on 4 at 7.6 uL/sec
	Dispensing 10.0 uL into A1 of Grenier BIO-ONE 384 Well Plate 138 µL on 3 at 7.6 uL/sec
	Aspirating 10.0 uL from A1 of NEST 12 Well Reservoir 15 mL on 4 at 7.6 uL/sec
	Dispensing 10.0 uL into A2 of Grenier BIO-ONE 384 Well Plate 138 µL on 3 at 7.6 uL/sec
	Aspirating 10.0 uL from A1 of NEST 12 Well Reservoir 15 mL on 4 at 7.6 uL/sec
	Dispensing 10.0 uL into A3 of Grenier BIO-ONE 384 Well Plate 138 µL on 3 at 7.6 uL/sec
	Aspirating 10.0 uL from A1 of NEST 12 Well Reservoir 15 mL on 4 at 7.6 uL/sec
	Dispensing 10.0 uL into A4 of Grenier BIO-ONE 384 Well Plate 138 µL on 3 at 7.6 uL/sec
	A

In [30]:
log, _bundle = simulate(open('../labware/grenierbioone_384_wellplate_138ul/test_grenierbioone_384_wellplate_138ul.py'))

print(format_runlog(log))

Picking up tip from A1 of Opentrons 96 Tip Rack 20 µL on 11
Homing pipette plunger on mount right
Moving to A1 of Grenier BIO-ONE 384 Well Plate 138 µL on 5
Logs from this command:
WARNING (labware): This method is deprecated. Please use 'from_center_cartesian' instead.
WARNING (labware): This method is deprecated. Please use 'from_center_cartesian' instead.
WARNING (labware): This method is deprecated. Please use 'from_center_cartesian' instead.
WARNING (labware): This method is deprecated. Please use 'from_center_cartesian' instead.
Pausing robot operation: If the position is accurate click 'resume.'
Moving to (142.73, 166.99, 14.4)
Pausing robot operation: If the position is accurate click 'resume.'
Moving to (146.53, 166.99, 14.4)
Pausing robot operation: If the position is accurate click 'resume.'
Moving to (144.63, 165.09, 14.4)
Pausing robot operation: If the position is accurate click 'resume.'
Moving to (144.63, 168.89000000000001, 14.4)
Pausing robot operation: If the positio

In [36]:
import os

ENV_SHARED_DATA_PATH = "OT_SHARED_DATA_PATH"

print(os.environ.get(ENV_SHARED_DATA_PATH))

None
